In [1]:
!pip install requests beautifulsoup4 colorthief selenium

  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa6703fcf4a79018f/attrs-23.2.0-py3-none-any.whl.metadata
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [8]:
pip install selenium webdriver-manager requests Pillow colorthief

  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b1/51/b5c11cf739ac4eecde611794a0ec9df420d0239d51e73bc19eb44f02b48b/webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install webcolors

Note: you may need to restart the kernel to use updated packages.


In [17]:
import io
import requests
from PIL import Image
from colorthief import ColorThief
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_image_from_url(url):
    response = requests.get(url)
    return Image.open(io.BytesIO(response.content))

def get_dominant_colors(image_data, num_colors=5):
    color_thief = ColorThief(image_data)
    palette = color_thief.get_palette(color_count=num_colors)
    return ['#{:02x}{:02x}{:02x}'.format(r, g, b) for r, g, b in palette]

def extract_logo(driver):
    selectors = [
        "img[src*='logo']",
        "img[alt*='logo']",
        "a[class*='logo'] img",
        "div[class*='logo'] img",
        "header img",
        "img" 
    ]
    for selector in selectors:
        try:
            logos = driver.find_elements(By.CSS_SELECTOR, selector)
            if logos:
                return logos[0].get_attribute('src')
        except Exception as e:
            print(f"Error finding logo with selector {selector}: {e}")
    print("Logo not found")
    return None

def extract_primary_colors(driver):
    screenshot = driver.get_screenshot_as_png()
    image_data = io.BytesIO(screenshot)
    return get_dominant_colors(image_data)

def extract_button_colors(driver):
    button_like_elements = driver.find_elements(By.CSS_SELECTOR, "button, a[class*='button'], div[class*='button'], span[class*='button']")
    colors = set()
    for element in button_like_elements:
        try:
            color = element.value_of_css_property('background-color')
            if color != 'rgba(0, 0, 0, 0)' and color != 'transparent':
                colors.add(color)
        except Exception as e:
            print(f"Error extracting button color: {e}")
    return [rgb_to_hex(color) for color in colors]

def rgb_to_hex(rgb):
    try:
        rgb = rgb.strip('rgba()').split(',')
        return '#{:02x}{:02x}{:02x}'.format(*map(int, rgb[:3]))
    except Exception as e:
        print(f"Error converting RGB to HEX: {e}")
        return None

def recommend_button_color(primary_colors):
    try:
        rgb_colors = [tuple(int(color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)) for color in primary_colors]
        avg_color = tuple(sum(color[i] for color in rgb_colors) // len(rgb_colors) for i in range(3))
        contrasting_color = tuple(255 - avg_color[i] for i in range(3))
        return '#{:02x}{:02x}{:02x}'.format(*contrasting_color)
    except Exception as e:
        print(f"Error recommending button color: {e}")
        return None

def main(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        logo_url = extract_logo(driver)
        primary_colors = extract_primary_colors(driver)
        button_colors = extract_button_colors(driver)
        recommended_button_color = recommend_button_color(primary_colors)

        # Outputs
        print("Logo URL:")
        print(logo_url)
        print("\nPrimary Colors:")
        print(primary_colors)
        print("\nButton Colors:")
        print(button_colors)
        print("\nRecommended Button Color:")
        print(recommended_button_color)

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()

if __name__ == '__main__':
    url = "https://www.amazon.com" 
    main(url)

Logo URL:
https://fls-na.amazon.com/1/batch/1/OP/ATVPDKIKX0DER:130-2828483-9032340:HFAV9BXSDCSYBNM3HY0A$uedata=s:%2Frd%2Fuedata%3Fstaticb%26id%3DHFAV9BXSDCSYBNM3HY0A:0

Primary Colors:
['#242e3a', '#f1e1d2', '#9e4e42', '#e6b229', '#b7997b']

Button Colors:
['#ffffff', '#ffd814']

Recommended Button Color:
#56779c
